In [1]:
using CSV
using DataFrames


In [2]:
path = "../../../data/soc-wiki-elec"

df = CSV.read(joinpath(path, "soc-wiki-elec.edges"), DataFrame, delim=" ", header=false, skipto = 2)

rename!(df, [:source, :target, :sign, :date])

first(df)

Row,source,target,sign,date
,Int64,Int64,Int64,Int64
1,3,30,1,1095171960


In [4]:
# get all edges
edges = [(df.source[i], df.target[i]) for i in 1:size(df, 1)]

edges[1:5]
print([length(unique(edges)), length(edges)])

# remove self loops
edges = [(u, v) for (u, v) in edges if u != v]
length(edges)

[103738, 107071]

107013

In [5]:
# check if for repeated edges the sign is the same

# get all edges
edges_sign = [(df.source[i], df.target[i], df.sign[i]) for i in 1:size(df, 1)]

# edges_sign = unique(edges_sign)

# remove self loops
edges_sign = [(u, v, s) for (u, v, s) in edges_sign if u != v]

# get the number of unique edges
length(edges_sign)

107013

We have some edges that change sign. Let's see if these edges form any triads

In [6]:
nrow(filter(x -> x.source != x.target, df)), nrow(df)

(107013, 107071)

In [7]:
# get list of all nodes
nodes = unique([[u for (u, v) in edges_sign]..., [v for (u, v) in edges_sign]...])

print(length(nodes))

# filter out self loops
df_filtered = filter(x -> x.source != x.target, df)

# get list of outgoing edges for each node by grouping by source
node_outs_gbf = groupby(df_filtered, :source)

# # filter out self loops
# node_outs_gbf = filter(x -> x[1, :source] != x[1, :target], node_outs_gbf)

# get list of outgoing edges for each node by collecting targets
node_outs = combine(node_outs_gbf, x -> [x[!, :target]])
rename!(node_outs, :x1 => :targets)

# create a dictionary with the outgoing edges for each node
node_outs_dict = Dict(zip(node_outs.source, unique.(node_outs.targets)))

7115

Dict{Int64, Vector{Int64}} with 6110 entries:
  4986 => [3898]
  7329 => [7295, 6832]
  4700 => [4261]
  4576 => [4578, 3447, 4448, 3897, 5463, 4977, 4530, 4536, 15, 3919]
  6073 => [5103]
  2288 => [2290]
  1703 => [230]
  7685 => [7683, 7757, 7908, 5179, 8037, 8124, 8174, 7391, 8198, 8219, 4401, 66…
  3406 => [3408, 3352, 3541, 4587]
  2841 => [72]
  2876 => [2877]
  7353 => [6832]
  185  => [214, 321, 2625, 1679]
  1090 => [1186, 943]
  2015 => [665]
  3293 => [6715, 6665, 6955, 840, 5289, 7908, 5614, 5969, 5605, 7620]
  1704 => [230]
  3220 => [2237]
  8042 => [6634]
  ⋮    => ⋮

In [8]:
# get transitive triads

triads = []
for (source, targets) in node_outs_dict
    for target in targets
        if target in keys(node_outs_dict)
            for target_target in node_outs_dict[target]
                if target_target in targets
                    append!(triads, [(source, target, target_target)])
                end
            end
        end
    end
end
length(triads), length(unique(triads))

(746515, 746515)

In [9]:
node_outs_dict_repeated = Dict(zip(node_outs.source, node_outs.targets))

# find edges that are repeated in node_outs_dict_repeated
repeated_edges = unique([(source, target) for (source, targets) in node_outs_dict_repeated for target in targets if sum(target .== targets) > 1])

3106-element Vector{Tuple{Int64, Int64}}:
 (3293, 6665)
 (1823, 2565)
 (1823, 6979)
 (637, 4037)
 (730, 989)
 (1050, 8290)
 (1050, 2651)
 (103, 271)
 (844, 857)
 (844, 1717)
 ⋮
 (1869, 1717)
 (1368, 3755)
 (1368, 2654)
 (3028, 2653)
 (3028, 3352)
 (3028, 4385)
 (3028, 4875)
 (3028, 4964)
 (3028, 737)

In [10]:
df_triads = DataFrame(triads)
rename!(df_triads, [:P, :O, :X]);

po_edges = [(df_triads.P[i], df_triads.O[i]) for i in 1:size(df_triads, 1)]
ox_edges = [(df_triads.O[i], df_triads.X[i]) for i in 1:size(df_triads, 1)]
px_edges = [(df_triads.P[i], df_triads.X[i]) for i in 1:size(df_triads, 1)];

In [11]:
# find number of triads containing repeated edges

affected_triads = []
for edge in repeated_edges
    append!(affected_triads, findall([edge] .== po_edges))
    # break
end
length(affected_triads), length(unique(affected_triads))

(55054, 55054)

In [12]:
# get sign of edges in triads. Take the last value of edge

df_triads[!, "po_sign"] = [edges_sign[findall([edge] .== edges)][end][3] for edge in po_edges]
df_triads[!, "ox_sign"] = [edges_sign[findall([edge] .== edges)][end][3] for edge in ox_edges]
df_triads[!, "px_sign"] = [edges_sign[findall([edge] .== edges)][end][3] for edge in px_edges]
first(df_triads, 5)

Row,P,O,X,po_sign,ox_sign,px_sign
,Int64,Int64,Int64,Int64,Int64,Int64
1,4576,3447,4448,1,1,1
2,4576,3447,5463,1,1,-1
3,4576,3447,4977,1,1,1
4,4576,3447,4530,1,1,1
5,4576,3447,3897,1,1,1


In [13]:
# save df_triads
CSV.write(joinpath(path, "soc-wiki-elec-triads.csv"), df_triads)

"../../../data/soc-wiki-elec/soc-wiki-elec-triads.csv"

In [5]:
df_triads = CSV.read(joinpath(path, "soc-wiki-elec-triads.csv"), DataFrame)

triads = [(df_triads.P[i], df_triads.O[i], df_triads.X[i]) for i in 1:size(df_triads, 1)]

746515-element Vector{Tuple{Int64, Int64, Int64}}:
 (4576, 3447, 4448)
 (4576, 3447, 5463)
 (4576, 3447, 4977)
 (4576, 3447, 4530)
 (4576, 3447, 3897)
 (4576, 4448, 4578)
 (4576, 4448, 4536)
 (4576, 4530, 3897)
 (4576, 4530, 15)
 (4576, 4536, 4578)
 ⋮
 (1711, 1697, 1855)
 (1711, 1697, 3136)
 (1711, 1697, 2328)
 (1711, 1211, 1956)
 (1711, 1211, 1151)
 (1711, 1211, 2328)
 (1711, 2328, 1956)
 (1711, 2328, 1211)
 (1711, 2328, 2210)

In [6]:
# identify triads that consist of the same edges

triads_ordered = sort.([[p, o, x] for (p, o, x) in triads])
triads_unique = unique(triads_ordered)

# create a dictionary with unique triads as keys and indices of their appearance as values
triads_dict = Dict(triads_unique .=> [findall(triads_ordered .== [triad]) for triad in triads_unique])



Dict{Vector{Int64}, Vector{Int64}} with 601555 entries:
  [1769, 2504, 3456] => [401752]
  [1542, 1628, 2193] => [370040, 625885]
  [1133, 2968, 3089] => [289925]
  [4632, 5189, 5760] => [310648]
  [457, 5776, 6913]  => [395368]
  [996, 1792, 8290]  => [624347]
  [2565, 2909, 5222] => [110895]
  [11, 1991, 2410]   => [687550]
  [2667, 5176, 5891] => [96199]
  [2145, 2760, 3615] => [549959]
  [3873, 5179, 5531] => [226922]
  [1210, 1744, 2654] => [367672]
  [1166, 1470, 2877] => [76235]
  [1608, 3352, 5288] => [208464]
  [1729, 2747, 3026] => [126602, 445214]
  [1029, 1319, 2625] => [671848]
  [2787, 2972, 3452] => [540996]
  [5472, 6458, 7073] => [741238]
  [1769, 2517, 4219] => [546185]
  ⋮                  => ⋮

In [29]:
using JLD2, FileIO

save(joinpath(path, "soc-wiki-elec-triads-dict.jld2"), "triads_dict", triads_dict)

In [45]:
# create dataframe from dict
df_triads_dict = DataFrame(triad = collect(keys(triads_dict)), inds = collect(values(triads_dict)),
                           count = length.(collect(values(triads_dict))))

# collect(keys(triads_dict))

Row,triad,inds,count
,Array…,Array…,Int64
1,"[1769, 2504, 3456]",[401752],1
2,"[1542, 1628, 2193]","[370040, 625885]",2
3,"[1133, 2968, 3089]",[289925],1
4,"[4632, 5189, 5760]",[310648],1
5,"[457, 5776, 6913]",[395368],1
6,"[996, 1792, 8290]",[624347],1
7,"[2565, 2909, 5222]",[110895],1
8,"[11, 1991, 2410]",[687550],1
9,"[2667, 5176, 5891]",[96199],1


In [50]:
println(df_triads_dict.triad[2])

triads[triads_dict[df_triads_dict.triad[2]]]

[1542, 1628, 2193]


2-element Vector{Tuple{Int64, Int64, Int64}}:
 (1628, 1542, 2193)
 (1542, 1628, 2193)

In [ ]:
using PyCall

i = 1
triad = df_triads_dict.triad[i]
inds = df_triads_dict.inds[i]

# get triad list
triad_list = triads[inds]

# get edges from triad list
edges_list = [[(triad[1], triad[2]), (triad[2], triad[3]), (triad[1], triad[3])] for triad in triad_list]
unique_edges = unique(edges_list)

py"""
import networkx as nx
"""

G = py"nx.DiGraph"(edges_list...)

# print(G.edges)

triad_census = py"nx.triadic_census"(G)

triad_census_triads = collect(keys(triad_census))

tc_ind = collect(values(triad_census)) .== 1

triad_census_triads[tc_ind][1]

# get signed triad type
sign_vals = values(df_triads[i, 4:6])

n_arr = zeros(Int, 8)

connections_values = sign_vals .== 1
n_arr[connections_values[1]*4 + connections_values[3]*2 + connections_values[2]+1] += 1

1

In [104]:
using PyCall

i = 2
triad = df_triads_dict.triad[i]
inds = df_triads_dict.inds[i]

# get triad list
triad_list = triads[inds]

# get edges from triad list
edges_list = [[(triad[1], triad[2]), (triad[2], triad[3]), (triad[1], triad[3])] for triad in triad_list]
edges_list = reduce(vcat, edges_list)
unique_edges = unique(edges_list)

py"""
import networkx as nx
"""

G = py"nx.DiGraph"([unique_edges...])

# print(G.edges)

triad_census = py"nx.triadic_census"(G)

triad_census_triads = collect(keys(triad_census))

tc_ind = collect(values(triad_census)) .== 1

triad_census_triads[tc_ind]

# get signed triad type
sign_vals_s = values(df_triads[inds, 4:6])
sign_vals_s = Matrix(sign_vals_s)
sign_vals_s = mapslices(x->[x], sign_vals_s, dims=2)[:]

n_arr = zeros(Int, 8)

for sign_vals in sign_vals_s
    connections_values = sign_vals .== 1
    n_arr[connections_values[1]*4 + connections_values[3]*2 + connections_values[2]+1] += 1
end
n_arr

8-element Vector{Int64}:
 0
 0
 0
 0
 0
 0
 0
 2

In [107]:
values(sign_vals)
mm = Matrix(sign_vals)
sign_vals_s = mapslices(x->[x], mm, dims=2)[:]

all(prod.(sign_vals_s) .== 1)

true

In [125]:
using PyCall
py"""
import networkx as nx
"""

G = py"nx.DiGraph"()

triad_census = py"nx.triadic_census"(G)
triad_census_triads = collect(keys(triad_census))

# signed triad types:
n_arr_dict = Dict( triad_census_triads .=> [zeros(Int, 8) for _ in 1:length(triad_census_triads)])
n_bal_dict = Dict( triad_census_triads .=> [0 for _ in 1:length(triad_census_triads)])
n_hier_dict = Dict( triad_census_triads .=> [0 for _ in 1:length(triad_census_triads)])
n_tot_dict = Dict( triad_census_triads .=> [0 for _ in 1:length(triad_census_triads)])

counter = 0
inform_after = 100000
for (triad, inds, count) in eachrow(df_triads_dict)
    triad_list = triads[inds]

    # get edges list from triad list
    edges_list = [[(triad[1], triad[2]), (triad[2], triad[3]), (triad[1], triad[3])] for triad in triad_list]
    edges_list = reduce(vcat, edges_list)
    unique_edges = unique(edges_list)

    # call Python from julia, use networkx to get triad census on subgraphs
    G = py"nx.DiGraph"([unique_edges...])
    triad_census = py"nx.triadic_census"(G)
    triad_census_triads = collect(keys(triad_census))
    tc_ind = collect(values(triad_census)) .== 1
    triad_type = triad_census_triads[tc_ind][1]

    # get signed triad type
    sign_vals_s = values(df_triads[inds, 4:6])
    sign_vals_s = Matrix(sign_vals_s)
    sign_vals_s = mapslices(x->[x], sign_vals_s, dims=2)[:]

    # n_arr = zeros(Int, 8)

    hier = true
    for sign_vals in sign_vals_s
        connections_values = sign_vals .== 1
        signed_triad_type = connections_values[1]*4 + connections_values[3]*2 + connections_values[2]+1
        n_arr_dict[triad_type][signed_triad_type] += 1

        if signed_triad_type == 3 || signed_triad_type == 6
            hier = false
        end
    end
    n_tot_dict[triad_type] += 1
    n_bal_dict[triad_type] += all(prod.(sign_vals_s) .== 1)
    n_hier_dict[triad_type] += hier

    # n_arr

    # println(triad_list, inds)
    counter += 1
    # if counter > 5000
    #     break
    # end
    if counter % inform_after == 0
        # print time
        # print()
        println("Processed $counter triads")
    end
end

Processed 100000 triads
Processed 200000 triads
Processed 300000 triads
Processed 400000 triads
Processed 500000 triads
Processed 600000 triads


In [126]:
# create dataframe from dicts

df_n_arr = DataFrame(triad = collect(keys(n_arr_dict)), n_arr = collect(values(n_arr_dict)),
                     n_bal = collect(values(n_bal_dict)), n_hier = collect(values(n_hier_dict)),
                     n_tot = collect(values(n_tot_dict)))

Row,triad,n_arr,n_bal,n_hier,n_tot
,String,Array…,Int64,Int64,Int64
1,030C,"[0, 0, 0, 0, 0, 0, 0, 0]",0,0,0
2,111U,"[0, 0, 0, 0, 0, 0, 0, 0]",0,0,0
3,120D,"[656, 2718, 804, 5957, 1529, 5232, 3536, 70680]",35682,39837,45556
4,120C,"[0, 0, 0, 0, 0, 0, 0, 0]",0,0,0
5,201,"[0, 0, 0, 0, 0, 0, 0, 0]",0,0,0
6,111D,"[0, 0, 0, 0, 0, 0, 0, 0]",0,0,0
7,210,"[148, 598, 344, 2666, 782, 2092, 2452, 36743]",11150,13025,15275
8,003,"[0, 0, 0, 0, 0, 0, 0, 0]",0,0,0
9,012,"[0, 0, 0, 0, 0, 0, 0, 0]",0,0,0


In [128]:
df_n_arr.per_bal = df_n_arr.n_bal ./ df_n_arr.n_tot
df_n_arr.per_hier = df_n_arr.n_hier ./ df_n_arr.n_tot
df_n_arr

Row,triad,n_arr,n_bal,n_hier,n_tot,per_bal,per_hier
,String,Array…,Int64,Int64,Int64,Float64,Float64
1,030C,"[0, 0, 0, 0, 0, 0, 0, 0]",0,0,0,NaN,NaN
2,111U,"[0, 0, 0, 0, 0, 0, 0, 0]",0,0,0,NaN,NaN
3,120D,"[656, 2718, 804, 5957, 1529, 5232, 3536, 70680]",35682,39837,45556,0.783256,0.874462
4,120C,"[0, 0, 0, 0, 0, 0, 0, 0]",0,0,0,NaN,NaN
5,201,"[0, 0, 0, 0, 0, 0, 0, 0]",0,0,0,NaN,NaN
6,111D,"[0, 0, 0, 0, 0, 0, 0, 0]",0,0,0,NaN,NaN
7,210,"[148, 598, 344, 2666, 782, 2092, 2452, 36743]",11150,13025,15275,0.729951,0.8527
8,003,"[0, 0, 0, 0, 0, 0, 0, 0]",0,0,0,NaN,NaN
9,012,"[0, 0, 0, 0, 0, 0, 0, 0]",0,0,0,NaN,NaN


In [ ]:
# save above dataframe
CSV.write(joinpath(path, "soc-wiki-elec-bal_hier_triads-types.csv"), df_n_arr)
CSV.write(joinpath("soc-wiki-elec-bal_hier_triads-types.csv"), df_n_arr)

# save JLD2
save(joinpath(path, "soc-wiki-elec-bal_hier_triads-types.jld2"), "df_n_arr", df_n_arr)
save("soc-wiki-elec-bal_hier_triads-types.jld2", "df_n_arr", df_n_arr)

In [1]:
joinpath(path)

UndefVarError: UndefVarError: `path` not defined